In [1]:
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1015. Fire Sufficient Similarity/3. Analyses/2. Data Processing/Output'
cur_date = '030125'

library(missForest)
library(readxl)
library(openxlsx)
library(tidyverse)
library(imputeLCMD)
library(factoextra)

# reading in files
ws_df = data.frame(Data = 'WS', read_excel("Input/Woodsmoke_Data_012825.xlsx", sheet = 2)) %>%
    select(-Sample_Number)
wf_df = data.frame(Data = 'WF', read_excel("Input/Wildfire_Data_012825.xlsx", sheet = 2)) %>%
    select(-Sample_Number)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some major limitations
(for example, it does not work reliably when
the number of variables exceeds 30),
an

NOTES TO SELF:
- change 'replicate' to 'instance' and 'chemical class' to 'class'

In [2]:
head(ws_df)
head(wf_df)

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,1263480,Burnet et al. 1990,NA,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WF,1289821,Liang et al. 2021,1,PAH,Volume,NA,1-(10-methylanthracen-9-yl)ethanone,5
2,WF,1289821,Liang et al. 2021,1,PAH,Volume,DTXSID50176885,1-Acenaphthenone,1
3,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
4,WF,1289739,Campos et al. 2019,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
5,WF,1289739,Campos et al. 2019,NA,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
6,WF,1289739,Campos et al. 2019,NA,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA


In [3]:
dim(ws_df)
dim(wf_df)

[1] 6607    9

[1] 4340    9

# Filter 1

Removing duplicate records for instances that weren't measured at all (MAR). 

In [4]:
`%notin%` <- Negate(`%in%`)

ws_df = ws_df %>%
    filter(Replicate %notin% NA)

wf_df = wf_df %>%
    filter(Replicate %notin% NA)

In [5]:
dim(ws_df)
dim(wf_df)

[1] 4203    9

[1] 2378    9

In [6]:
# combining dfs
combined_df = rbind(ws_df, wf_df)
head(combined_df)

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,267091,Forchhammer et al. 2012,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


In [7]:
length(unique(combined_df$Name))

[1] 192

192 unique variables.

# Variable Background Filters

Determining what variables have missing data for all variables delineated based on the dfs the data will be split into and analyzed. 

1. The data needs to have at least two experimental (ie. measured or ND) value in both WS and WF data within each metric (ie. weight or volume) and study.
2. A variable (ie. `Name` which represents a chemical or metal) will be removed if < 50% of its experimental data points were either measurable values and/or specified as non-detects. In other words, chemicals were excluded if they were NAs (not evaluated) across <= 50% of the samples.

In [8]:
sample_type_presence_df = combined_df %>%
    # if an experimental value is there count it as being "present"
    mutate(count = ifelse(Value %notin% c('NA', 'ND'), 1, 0)) %>%
    # determining which have at least one experimental value within each sample type
    group_by(HAWC_ID, Metric, Name) %>%
    # summing the number of experimental records for each variable?????????
    reframe(Variable_Presence_Percentage = (sum(count)/n()) * 100) %>%
    filter(Variable_Presence_Percentage >= 50)

head(sample_type_presence_df)

HAWC_ID,Metric,Name,Variable_Presence_Percentage
<dbl>,<chr>,<chr>,<dbl>
266980,Volume,Acetaldehyde,100
266980,Volume,Benzene,100
266980,Volume,Benzo(GHI)perylene,100
266980,Volume,Benzo(a)pyrene,100
266980,Volume,Benzo(b)fluoranthene,100
266980,Volume,Benzo(k)fluoranthene,100


278 data records had at least two experimental values and were retained. 

In [9]:
# filtering the original df
filter1_df = inner_join(sample_type_presence_df[,1:3], combined_df)
head(filter1_df)

Joining with `by = join_by(HAWC_ID, Metric, Name)`


HAWC_ID,Metric,Name,Data,Study,Replicate,Chemical_Class,DTXSID,Value
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
266980,Volume,Acetaldehyde,WS,Stockfelt et al. 2012,1,VOC,DTXSID5039224,71000
266980,Volume,Acetaldehyde,WS,Stockfelt et al. 2012,2,VOC,DTXSID5039224,37000
266980,Volume,Benzene,WS,Stockfelt et al. 2012,1,VOC,DTXSID3039242,33000
266980,Volume,Benzene,WS,Stockfelt et al. 2012,2,VOC,DTXSID3039242,21000
266980,Volume,Benzo(GHI)perylene,WS,Stockfelt et al. 2012,1,PAH,DTXSID5023908,41
266980,Volume,Benzo(GHI)perylene,WS,Stockfelt et al. 2012,2,PAH,DTXSID5023908,11


In [10]:
dim(combined_df)
dim(filter1_df)

[1] 6581    9

[1] 4189    9

Started with 6581 records, 1363 were removed, leaving 5145. 

Now that each sample type (WS of WF) has at least one experimental value, we'll see if there are at least 50% of data between the sample types.

In [11]:
variable_presence_df = filter1_df %>%
    # if the value isn't MAR count it as being present
    mutate(count = ifelse(Value != 'NA', 1, 0)) %>%
    group_by(Metric, Name) %>%
    # calculating the percentage of variables with data overall
    reframe(Variable_Presence_Percentage = (sum(count)/n()) * 100) %>%
    arrange(-Variable_Presence_Percentage)

# viewing data that passed the filter
keep_variables_df = variable_presence_df %>%
     filter(Variable_Presence_Percentage >= 50) %>%
     unique()

head(keep_variables_df)

Metric,Name,Variable_Presence_Percentage
<chr>,<chr>,<dbl>
Volume,"1,2,4-Trimethylbenzene",100
Volume,"1,3,5-Trimethylbenzene",100
Volume,"1,3-Dihydroxynaphthalene",100
Volume,"1,4-Dichloro-2-butene, cis",100
Volume,"1,4-Dichlorobenzene",100
Volume,"1,8-Dihydroxynaphthalene",100


In [13]:
# only keeping records that passed the background filter
filter2_df = inner_join(keep_variables_df[,1:2], filter1_df) %>%
    # metals don't have a DTXSID, so making that col their name
    mutate(DTXSID = ifelse(DTXSID != 'NA', DTXSID, Name)) 
    

head(filter2_df)

Joining with `by = join_by(Metric, Name)`


Metric,Name,HAWC_ID,Data,Study,Replicate,Chemical_Class,DTXSID,Value
<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
Volume,"1,2,4-Trimethylbenzene",1289926,WF,Wang et al. 2024,1,VOC,DTXSID6021402,730
Volume,"1,2,4-Trimethylbenzene",1306371,WF,Ketcherside et al. 2024,1,VOC,DTXSID6021402,130
Volume,"1,3,5-Trimethylbenzene",1289926,WF,Wang et al. 2024,1,VOC,DTXSID6026797,330
Volume,"1,3,5-Trimethylbenzene",1306371,WF,Ketcherside et al. 2024,1,VOC,DTXSID6026797,110
Volume,"1,3-Dihydroxynaphthalene",1289821,WF,Liang et al. 2021,1,PAH,DTXSID40456587,6
Volume,"1,4-Dichloro-2-butene, cis",1289926,WF,Wang et al. 2024,1,VOC,DTXSID3027405,230


In [14]:
dim(filter2_df)

[1] 4189    9

An additional 408 records were removed, leaving 4882.

# Second Variable Background Filter

In the intial filter, samples were combined. However, this time variables (ie. metal or chemical) will be split based on their `Metric` (ie. volume or weight) and then retained if that variable in found in both woodsmoke and wildfire samples within volume or weight samples.

In [15]:
split_filtered_df = filter2_df %>%
    group_by(Data, Metric) %>%
    group_split

split_ws_vol_df = split_filtered_df[[3]]
split_ws_weight_df = split_filtered_df[[4]]
split_wf_vol_df = split_filtered_df[[1]]
split_wf_weight_df = split_filtered_df[[2]]

In [16]:
# seeing how many unique variables are in each df and if they're consistent in each file
length(unique(split_ws_vol_df$Name))
length(unique(split_wf_vol_df$Name))
length(unique(split_ws_weight_df$Name))
length(unique(split_wf_weight_df$Name))

[1] 98

[1] 68

[1] 94

[1] 70

In [17]:
# they're not so first getting variables that are in weight or volume samples
consistent_wf_vol_df = split_wf_vol_df %>%
    filter(Name %in% unique(split_ws_vol_df$Name))
consistent_ws_vol_df = split_ws_vol_df %>%
    filter(Name %in% consistent_wf_vol_df$Name)
consistent_wf_weight_df = split_wf_weight_df %>%
    filter(Name %in% unique(split_ws_weight_df$Name))
consistent_ws_weight_df = split_ws_weight_df %>%
    filter(Name %in% consistent_wf_weight_df$Name)

length(unique(consistent_wf_vol_df$Name))
length(unique(consistent_ws_vol_df$Name))
length(unique(consistent_wf_weight_df$Name))
length(unique(consistent_ws_weight_df$Name))

[1] 33

[1] 33

[1] 34

[1] 34

There were 80, 47, 73 and 61 woodsmoke weight, woodsmoke volume, wildfire weight, and wildfire volume samples, respectively. 26 variables were common between volume samples and 30 were common between weight records and will be retained.

In [18]:
# recombining data
vol_df = rbind(consistent_ws_vol_df, consistent_wf_vol_df)
weight_df = rbind(consistent_ws_weight_df, consistent_wf_weight_df)

head(vol_df)

Metric,Name,HAWC_ID,Data,Study,Replicate,Chemical_Class,DTXSID,Value
<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
Volume,Acenaphthylene,429445,WS,Rajput 2010,1,PAH,DTXSID3023845,53500
Volume,Acenaphthylene,1263480,WS,Burnet et al. 1990,1,PAH,DTXSID3023845,6187000
Volume,Acenaphthylene,1263480,WS,Burnet et al. 1990,2,PAH,DTXSID3023845,18890500
Volume,Acenaphthylene,1263480,WS,Burnet et al. 1990,3,PAH,DTXSID3023845,7806000
Volume,Acenaphthylene,1263484,WS,Leese et al. 1989,1,PAH,DTXSID3023845,1100000
Volume,Acenaphthylene,1263484,WS,Leese et al. 1989,2,PAH,DTXSID3023845,2800000


# QRILC Imputation

Imputing non-detect data using QRILC didn't work, so I'm just replacing the non-detect data with zeros.

In [21]:
# head(weight_df %>%
#     filter(Value == 'ND'))

Metric,Name,Data,HAWC_ID,Study,Replicate,Chemical_Class,DTXSID,Value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
Weight,Fluoranthene,WS,1098462,Niu et al. 2023,3,PAH,DTXSID3024104,ND
Weight,Phenanthrene,WS,1098462,Niu et al. 2023,3,PAH,DTXSID6024254,ND
Weight,Phenanthrene,WS,914540,Verma et al. 2021,7,PAH,DTXSID6024254,ND
Weight,Phenanthrene,WS,914540,Verma et al. 2021,8,PAH,DTXSID6024254,ND
Weight,Ni,WS,822010,Farina et al. 2019,1,Metal,Ni,ND
Weight,Ni,WS,299223,Kasurinen et al. 2015,2,Metal,Ni,ND


Only the weight dataframe had non-detect values representing MNAR data. All MAR data will be imputed using random forest (RF).

In [54]:
mar_weight_df = weight_df %>%
    filter(Value == 'NA')

preimputed_df = anti_join(weight_df, mar_weight_df) 
preimputed_df$Value = as.numeric(preimputed_df$Value)

# reordering
preimputed_df = preimputed_df[,c(3,1,4:8,2,9)]

head(preimputed_df)

Joining with `by = join_by(Metric, Name, HAWC_ID, Data, Study, Replicate,
Chemical_Class, DTXSID, Value)`
Warning message:
“NAs introduced by coercion”


HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
821855,Weight,WS,Erlandsson et al. 2020,1,PAH,DTXSID2060383,"2,3-Dimethylnaphthalene",0.05
821855,Weight,WS,Erlandsson et al. 2020,2,PAH,DTXSID2060383,"2,3-Dimethylnaphthalene",0.05
1257056,Weight,WS,McCarrick et al. 2024,1,PAH,DTXSID2060383,"2,3-Dimethylnaphthalene",0.04
821855,Weight,WS,Erlandsson et al. 2020,1,PAH,DTXSID8074819,2-Methylchrysene,7.40
821855,Weight,WS,Erlandsson et al. 2020,2,PAH,DTXSID8074819,2-Methylchrysene,12.10
1257056,Weight,WS,McCarrick et al. 2024,1,PAH,DTXSID8074819,2-Methylchrysene,15.45


WORDS ABOUT CHANGING VALUES MANUALLY!!

In [55]:
# will impute within each study
study_id = unique(preimputed_df$HAWC_ID)

preimputed_df = preimputed_df %>%
    mutate(Value = ifelse(HAWC_ID %in% c(821855) & is.na(Value) & Name == 'Acenaphthene', 0.1, Value))

In [56]:
QRILC_imputation = function(dataset){
      # """
    # Creating a quantile normalization function to normalize each sample.
    # :param (input): exposed and unexposed (vehicle) dfs
    # :output: 1 quantile normalized df
    # """
    #ADD WORDS
    wider_dataset = dataset%>%#[,c(1:2,7,9)] %>%
        # removing these cols temporarily 
        select(-c("Study", "HAWC_ID", "Chemical_Class", "Name")) %>%
        pivot_wider(names_from = DTXSID, values_from = Value)
    
    index_of_last_variable = length(colnames(wider_dataset))

    # normalizing data since that what the QRILC function wants
    # had to pseudo log transform to prevent Inf values
    QRILC_prep = wider_dataset[,6:dim(wider_dataset)[2]] %>%
         mutate_all(., function(x) log10(x + 1)) %>%
         as.matrix()
                    
    imputed_QRILC_object = impute.QRILC(QRILC_prep, tune.sigma = 0.1)
    QRILC_log10_df = data.frame(imputed_QRILC_object[1]) 
    
    # converting back the original scale
    QRILC_df = QRILC_log10_df %>%
        mutate_all(., function(x) 10^x - 1)
     
    imputed_dataset = data.frame(cbind(unique(dataset[,1:5]), QRILC_df)) %>%
         pivot_longer(cols = 6:dim(wider_dataset)[2], names_to = "Variable", values_to = "Value")
    
    return(imputed_dataset)

}
     

In [59]:
for (i in 1:length(study_id)){

    filtered_df = preimputed_df %>%
        filter(HAWC_ID == study_id[i])

    print(head(filtered_df))
    # calling fn
    imp_df = QRILC_imputation(filtered_df)
    
}

# A tibble: 6 × 9
  HAWC_ID Metric Data  Study        Replicate Chemical_Class DTXSID Name   Value
    <dbl> <chr>  <chr> <chr>            <dbl> <chr>          <chr>  <chr>  <dbl>
1 1257056 Weight WS    McCarrick e…         1 PAH            DTXSI… 2,3-…   0.04
2 1257056 Weight WS    McCarrick e…         1 PAH            DTXSI… 2-Me…  15.4 
3 1257056 Weight WS    McCarrick e…         1 PAH            DTXSI… Biph…   0.08
4 1257056 Weight WS    McCarrick e…         1 PAH            DTXSI… Fluo… 215.  
5 1257056 Weight WS    McCarrick e…         1 PAH            DTXSI… Phen…  62.4 
6 1257056 Weight WS    McCarrick e…         1 PAH            DTXSI… 1-Me…  41.9 


Warning message in qnorm((pNAs + 0.001), mean = mean.CDD, sd = sd.CDD):
“NaNs produced”


ERROR: Error in checkTmvArgs(mean, sigma, lower, upper): ‘upper’ not specified or contains NA


In [70]:
preimputed_df %>%
        filter(HAWC_ID == study_id[10]) 

HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267127,Weight,WS,Bølling et al. 2012,1,PAH,DTXSID4075455,Benzo(GHI)fluoranthene,168
267127,Weight,WS,Bølling et al. 2012,2,PAH,DTXSID4075455,Benzo(GHI)fluoranthene,336
267127,Weight,WS,Bølling et al. 2012,3,PAH,DTXSID4075455,Benzo(GHI)fluoranthene,90
267127,Weight,WS,Bølling et al. 2012,1,PAH,DTXSID4075459,Benzo(c)phenanthrene,10
267127,Weight,WS,Bølling et al. 2012,2,PAH,DTXSID4075459,Benzo(c)phenanthrene,32
267127,Weight,WS,Bølling et al. 2012,1,PAH,DTXSID3024104,Fluoranthene,73
267127,Weight,WS,Bølling et al. 2012,2,PAH,DTXSID3024104,Fluoranthene,105
267127,Weight,WS,Bølling et al. 2012,3,PAH,DTXSID3024104,Fluoranthene,59
267127,Weight,WS,Bølling et al. 2012,1,PAH,DTXSID6024254,Phenanthrene,16


In [21]:
test = preimputed_df %>%
    filter(Name == 'Fluoranthene', HAWC_ID == '1098462')
test

test1 = preimputed_df %>%
    filter(Chemical_Class == 'Metal')

# imputing within each study
study_id = unique(preimputed_df$HAWC_ID)
test2 = preimputed_df %>%
    filter(HAWC_ID == study_id[8])
head(test2)

HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1098462,Weight,WS,Niu et al. 2023,1,PAH,DTXSID3024104,Fluoranthene,0.21
1098462,Weight,WS,Niu et al. 2023,2,PAH,DTXSID3024104,Fluoranthene,0.18
1098462,Weight,WS,Niu et al. 2023,3,PAH,DTXSID3024104,Fluoranthene,NA


HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1098462,Weight,WS,Niu et al. 2023,1,Metal,Al,Al,1.72
1098462,Weight,WS,Niu et al. 2023,2,Metal,Al,Al,3.02
1098462,Weight,WS,Niu et al. 2023,3,Metal,Al,Al,3.09
1098462,Weight,WS,Niu et al. 2023,1,Metal,Ba,Ba,2.40
1098462,Weight,WS,Niu et al. 2023,2,Metal,Ba,Ba,1.65
1098462,Weight,WS,Niu et al. 2023,3,Metal,Ba,Ba,1.55


In [22]:
idk = test2 %>%
    select(-c("Study", "HAWC_ID", "Chemical_Class", "Name")) %>%
    pivot_wider(names_from = DTXSID, values_from = Value) 
head(idk)
idk2 = idk[,4:11] %>%
    mutate_all(., function(x) log10(x + 1)) %>% as.matrix()

idk2

impute.QRILC(idk2[,c(1:6,8)], tune.sigma = 0.1)

Metric,Data,Replicate,Al,Ba,Co,Cu,K,Mg,Si,⋯,Zn,DTXSID3024104,Ni,Mn,Cr,Ca,Fe,Rb,DTXSID6024254,DTXSID3021774
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Weight,WS,1,1.72,2.40,0.01,0.43,174.21,1.75,4.21,⋯,3.12,0.21,0.03,0.24,0.19,1.65,1.32,0.15,0.09,NA
Weight,WS,2,3.02,1.65,0.02,0.35,45.67,0.68,3.89,⋯,1.32,0.18,0.04,0.36,0.15,0.40,1.13,NA,0.01,0.09
Weight,WS,3,3.09,1.55,NA,0.27,285.03,0.43,1.58,⋯,1.79,NA,0.05,0.21,0.10,0.98,0.53,0.10,NA,0.05


Al,Ba,Co,Cu,K,Mg,Si,Ti
0.4345689,0.5314789,0.004321374,0.1553360,2.243559,0.4393327,0.7168377,0.02118930
0.6042261,0.4232459,0.008600172,0.1303338,1.669038,0.2253093,0.6893089,0.01283722
0.6117233,0.4065402,NA,0.1038037,2.456412,0.1553360,0.4116197,0.01283722


[[1]]
            Al        Ba          Co        Cu        K        Mg         Ti
[1,] 0.4345689 0.5314789 0.004321374 0.1553360 2.243559 0.4393327 0.02118930
[2,] 0.6042261 0.4232459 0.008600172 0.1303338 1.669038 0.2253093 0.01283722
[3,] 0.6117233 0.4065402 0.004640072 0.1038037 2.456412 0.1553360 0.01283722

[[2]]
[[2]][[1]]

Call:
lm(formula = q.curr.sample ~ q.normal)

Coefficients:
(Intercept)     q.normal  
    0.56429      0.05073  


[[2]][[2]]

Call:
lm(formula = q.curr.sample ~ q.normal)

Coefficients:
(Intercept)     q.normal  
    0.44652      0.03446  


[[2]][[3]]

Call:
lm(formula = q.curr.sample ~ q.normal)

Coefficients:
(Intercept)     q.normal  
   0.005481     0.001843  


[[2]][[4]]

Call:
lm(formula = q.curr.sample ~ q.normal)

Coefficients:
(Intercept)     q.normal  
    0.13012      0.01447  


[[2]][[5]]

Call:
lm(formula = q.curr.sample ~ q.normal)

Coefficients:
(Intercept)     q.normal  
     2.1558       0.2232  


[[2]][[6]]

Call:
lm(formula = q.curr.s

In [52]:
weight_df %>%
    filter(Name == 'Acenaphthene')
    # group_by(Name) %>%
    # summarize(n())

Metric,Name,HAWC_ID,Data,Study,Replicate,Chemical_Class,DTXSID,Value
<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
Weight,Acenaphthene,271638,WS,Jalava et al. 2012,1,PAH,DTXSID3021774,132.7
Weight,Acenaphthene,271638,WS,Jalava et al. 2012,2,PAH,DTXSID3021774,2.9
Weight,Acenaphthene,271638,WS,Jalava et al. 2012,3,PAH,DTXSID3021774,51.3
Weight,Acenaphthene,271638,WS,Jalava et al. 2012,4,PAH,DTXSID3021774,ND
Weight,Acenaphthene,271638,WS,Jalava et al. 2012,5,PAH,DTXSID3021774,0.8
Weight,Acenaphthene,271638,WS,Jalava et al. 2012,6,PAH,DTXSID3021774,ND
Weight,Acenaphthene,271638,WS,Jalava et al. 2012,7,PAH,DTXSID3021774,ND
Weight,Acenaphthene,271769,WS,Uski et al. 2012,1,PAH,DTXSID3021774,130
Weight,Acenaphthene,271769,WS,Uski et al. 2012,2,PAH,DTXSID3021774,2.9


# RF Imputation

In [23]:
preimputed_weight_df = weight_df %>%
    mutate(Value = ifelse(Value == 'ND', '0', Value)) %>%
    group_by(Data, HAWC_ID, Name) %>%
    mutate(Avg_Value = mean(as.numeric(Value, na.rm = TRUE))) %>%
    select(-c("Replicate", "Value")) %>%
    unique()
    # creating sample ids
    #unite("SampleID", c(colnames(weight_df)[c(3,4,6)]), sep = "_")

#preimputed_weight_df$Value = as.numeric(preimputed_weight_df$Value)

head(preimputed_weight_df)

Warning message:
“There were 220 warnings in `mutate()`.
The first warning was:
ℹ In argument: `Avg_Value = mean(as.numeric(Value, na.rm = TRUE))`.
ℹ In group 244: `Data = "WS"`, `HAWC_ID = 267105`, `Name = "Acenaphthene"`.
Caused by warning in `mean()`:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 219 remaining warnings.”


Metric,Name,Data,HAWC_ID,Study,Chemical_Class,DTXSID,Avg_Value
<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
Weight,"2,3-Dimethylnaphthalene",WS,821855,Erlandsson et al. 2020,PAH,DTXSID2060383,0.05
Weight,"2,3-Dimethylnaphthalene",WS,1257056,McCarrick et al. 2024,PAH,DTXSID2060383,0.04
Weight,2-Methylchrysene,WS,821855,Erlandsson et al. 2020,PAH,DTXSID8074819,9.75
Weight,2-Methylchrysene,WS,1257056,McCarrick et al. 2024,PAH,DTXSID8074819,15.45
Weight,2-Methylphenanthrene,WS,267105,Danielsen et al. 2011,PAH,DTXSID4020878,23.50
Weight,2-Methylphenanthrene,WS,821855,Erlandsson et al. 2020,PAH,DTXSID4020878,3.50


In [31]:
ncol(preimputed_weight_df)

[1] 8

In [58]:
random_forest_imputation = function(dataset){
    # ADD WORDS
    wider_dataset = dataset %>%
        pivot_wider(names_from = DTXSID, values_from = Avg_Value)
    preimputed_df = wider_dataset[,8:ncol(wider_dataset)]
    # had to drop rows (studies) that only had missing data
    df = preimputed_df[rowSums(is.na(preimputed_df)) != ncol(preimputed_df), ]
    
    imputed_RF_object = missForest(as.matrix(df))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(wider_dataset[,1:7], imputed_RF_df)) %>%
        pivot_longer(cols = 8:ncol(wider_dataset), names_to = "DTXSID", values_to = "Value")
    
    return(imputed_final_df)
}


# calling fn
a = random_forest_imputation(preimputed_weight_df %>% filter(Data == 'WS'))

Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response h

ERROR: Error in if (n == 0) stop("data (x) has 0 rows"): argument is of length zero


In [48]:
l = data.frame(a[,8:37])
dim(l)

[1] 698  30

In [49]:
l[rowSums(is.na(l)) != ncol(l), ]

,DTXSID8074819,DTXSID4020878,B,DTXSID4075459,DTXSID4020161,Cu,DTXSID0047741,K,Mg,Zn,⋯,Si,Sr,Ti,Ba,Rb,DTXSID9059757,Li,Bi,Th,DTXSID1025649
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,9.75,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,15.45,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,NA,23.500,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,NA,3.500,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
7,NA,5.661,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
8,NA,NA,288,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,NA,NA,NA,21.00000,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
10,NA,NA,NA,235.42857,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
11,NA,NA,NA,0.45000,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [55]:
l

DTXSID8074819,DTXSID4020878,B,DTXSID4075459,DTXSID4020161,Cu,DTXSID0047741,K,Mg,Zn,⋯,Si,Sr,Ti,Ba,Rb,DTXSID9059757,Li,Bi,Th,DTXSID1025649
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9.75,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
15.45,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,23.500,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,3.500,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,5.661,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,288,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,21.00000,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [20]:
# # exporting
# write.xlsx(processed_df, paste0(Output,"/", "Processed_Data_", cur_date, ".xlsx"), 
#            rowNames = FALSE)